# Univariate Feature Selection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_09/VL09_Univariate_Feature_Selection.ipynb)

**Lernziel:** Verstehen und anwenden von univariaten Feature Selection Methoden

**Methode:**
- 📊 **F-Test (ANOVA):** Für numerische Features und kategoriale Zielvariable

**Szenario:** Welche Features sind am wichtigsten für die Vorhersage?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Für bessere Plots
plt.style.use('default')
sns.set_palette("husl")

print("📚 Libraries geladen!")

## 1. Datensatz erstellen

Wir erstellen einen synthetischen Datensatz mit verschiedenen Feature-Typen:

In [ ]:
# Synthetischen Datensatz erstellen
np.random.seed(42)

# Basis-Features mit make_classification
X_base, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=5,
    n_redundant=2,
    n_clusters_per_class=1,
    random_state=42
)

# Zusätzliche kategoriale Features erstellen
n_samples = X_base.shape[0]

# Kategoriale Features (für Chi-Square Test)
kategorie_A = np.random.choice(['low', 'medium', 'high'], n_samples, p=[0.3, 0.4, 0.3])
kategorie_B = np.random.choice(['type1', 'type2', 'type3'], n_samples)

# Eine kategorie die mit y korreliert (informativ)
kategorie_informativ = np.where(y == 1, 
                                np.random.choice(['A', 'B'], n_samples, p=[0.8, 0.2]),
                                np.random.choice(['A', 'B'], n_samples, p=[0.2, 0.8]))

# Noise Features hinzufügen
noise_features = np.random.randn(n_samples, 5)

# Alles zu einem DataFrame kombinieren
df = pd.DataFrame(X_base, columns=[f'numeric_{i}' for i in range(10)])
df['kategorie_A'] = kategorie_A
df['kategorie_B'] = kategorie_B
df['kategorie_informativ'] = kategorie_informativ

# Noise features hinzufügen
for i in range(5):
    df[f'noise_{i}'] = noise_features[:, i]

df['target'] = y

print(f"Dataset erstellt: {df.shape}")
print(f"Features: {df.columns.tolist()}")
print(f"\nTarget-Verteilung:")
print(df['target'].value_counts())

df.head()

## 2. F-Test (ANOVA) für numerische Features

Der **F-Test** misst, ob numerische Features signifikant unterschiedliche Mittelwerte zwischen den Klassen haben.

In [ ]:
# Numerische Features extrahieren
numeric_features = [col for col in df.columns if col.startswith(('numeric_', 'noise_'))]
X_numeric = df[numeric_features]
y_target = df['target']

print(f"Numerische Features: {len(numeric_features)}")
print(f"Features: {numeric_features}")

# F-Test durchführen
f_selector = SelectKBest(score_func=f_classif, k='all')
f_selector.fit(X_numeric, y_target)

# Ergebnisse sammeln
f_scores = f_selector.scores_
f_pvalues = f_selector.pvalues_

# Ergebnisse in DataFrame
f_results = pd.DataFrame({
    'Feature': numeric_features,
    'F_Score': f_scores,
    'P_Value': f_pvalues,
    'Significant': f_pvalues < 0.05
})

# Nach F-Score sortieren
f_results = f_results.sort_values('F_Score', ascending=False)

print("\n🔍 F-Test Ergebnisse (numerische Features):")
print(f_results)

In [ ]:
# F-Test Ergebnisse visualisieren
plt.figure(figsize=(12, 6))

# Subplot 1: F-Scores
plt.subplot(1, 2, 1)
colors = ['green' if sig else 'red' for sig in f_results['Significant']]
bars = plt.bar(range(len(f_results)), f_results['F_Score'], color=colors, alpha=0.7)
plt.xticks(range(len(f_results)), f_results['Feature'], rotation=45, ha='right')
plt.ylabel('F-Score')
plt.title('F-Test Scores (numerische Features)')
plt.grid(True, alpha=0.3)

# Legende
import matplotlib.patches as mpatches
green_patch = mpatches.Patch(color='green', alpha=0.7, label='Signifikant (p < 0.05)')
red_patch = mpatches.Patch(color='red', alpha=0.7, label='Nicht signifikant')
plt.legend(handles=[green_patch, red_patch])

# Subplot 2: P-Values
plt.subplot(1, 2, 2)
plt.bar(range(len(f_results)), -np.log10(f_results['P_Value']), color=colors, alpha=0.7)
plt.xticks(range(len(f_results)), f_results['Feature'], rotation=45, ha='right')
plt.ylabel('-log10(P-Value)')
plt.title('Statistische Signifikanz')
plt.axhline(y=-np.log10(0.05), color='black', linestyle='--', alpha=0.7, label='p = 0.05')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Interpretation:")
print(f"- Grüne Balken: Statistisch signifikante Features (p < 0.05)")
print(f"- Rote Balken: Nicht signifikante Features")
print(f"- Höhere F-Scores = bessere Trennung zwischen Klassen")

## Zusammenfassung: F-Test für Feature Selection

### Was haben wir gelernt?

**F-Test (ANOVA):**
- 📊 Misst, ob numerische Features signifikant unterschiedliche Mittelwerte zwischen Klassen haben
- 🎯 Höhere F-Scores = bessere Trennung zwischen Klassen  
- 📈 P-Werte < 0.05 = statistisch signifikant

**Praktische Anwendung:**
- ✅ Gut für: Numerische Features → Kategoriale Zielvariable
- 🔍 Hilft bei: Identifikation der wichtigsten Features
- ⚠️ Limitierung: Betrachtet Features einzeln (univariat), nicht Kombinationen

**Next Steps:**
- Chi-Square Test für kategoriale Features
- Multivariate Feature Selection Methoden
- Anwendung in echten ML-Pipelines